In [13]:
parsed_file = "test_file"
word_occurence_map = {}

with open(parsed_file) as inp:
    prev_line, prev_count = '', 0
    for line in inp:
        curr_line = "\t".join(line.strip().split("\t")[:-1])
        if curr_line == prev_line or (not prev_line):
            prev_count += int(line.strip().split("\t")[-1])
        else:
            print (curr_line)
            x, y, path = prev_line.split('\t')
            key = str(x) + '_' + str(y)
            current = path + ":" + str(prev_count)
            if key in word_occurence_map:
                pastkeys = word_occurence_map[key]
                current =  pastkeys + ',' + current
            word_occurence_map[key] = current
            prev_count = 1
        prev_line = curr_line
    x, y, path = prev_line.split('\t')
    key = str(x) + '_' + str(y)
    current = path + ":" + str(prev_count)
    if key in word_occurence_map:
        pastkeys = word_occurence_map[key]
        current =  pastkeys + ',' + current
    word_occurence_map[key] = current
    


19193456	23452389	9668810
663726	9048846	4107215
6178566	6097931	8733868
8184873	13389052	6536081
5933740	15640897	8593453
1837912	2946514	9443662
16864707	22008180	11071817
13583494	15294065	657327
18679314	8042043	11954734
18679314	8042040	11954736
18679314	8042043	11954739


In [23]:
import sys, shelve, picklw
file = "damru"
p = pickle.load(open(file, 'rb'))
db_file = ".".join(file.split(".")[:-1]) + ".db" 
with shelve.open(db_file, 'c') as db:
    for k in p:
        try:
            p[k] = p[k].decode("utf-8")
        except:
            pass
        try:
            k = k.decode("utf-8")
        except:
            pass

        if k and p[k]:
            db[str(k)] = str(p[k])
print ("db_file", db_file, "done")

KeyboardInterrupt: 

In [32]:
control = """access control regulation control compliant organization	control	hypernym
access control	control	hypernym
access regulation	control	hypernym
antivirus	control	hypernym
antivirus control	control	hypernym
antivirus control compliant mobile device	control	hypernym
mobile antivirus	control	hypernym
antivirus control compliant mobile	control	hypernym
antivirus	software	hypernym
antivirus software	control	hypernym
antivirus software control compliant computer	control	hypernym
computer antivirus	control	hypernym
anti virus	control	hypernym
anti virus control	control	hypernym
anti virus	software	hypernym
anti virus software	control	hypernym
anti virus software control compliant computer	control	hypernym
mobile anti virus	control	hypernym
computer anti virus	control	hypernym
air conditioning control compliant server room	control	hypernym
air conditioning control	control	hypernym
air conditioning server room	control	hypernym
server room	control	hypernym
access regulation control compliant building	control	hypernym
building	control	hypernym
access regulation	control	hypernym
access control	control	hypernym
access control regulation control compliant organization	control	hypernym
access control compliant organization	control	hypernym
regulation control compliant organization	control	hypernym
regulation control	control	hypernym
audit logging	control	hypernym
audit logging control	control	hypernym
audit logging and monitoring	control	hypernym
audit logging and monitoring control compliant organization	control	hypernym
audit logging control compliant organization	control	hypernym
audit monitoring control compliant organization	control	hypernym
audit organization	control	hypernym
application evaluation	control	hypernym
application evaluation control	control	hypernym
application evaluation control compliant organization	control	hypernym
assets control	control	hypernym
assets control compliant organization	control	hypernym
appropriate contacts	control	hypernym
appropriate contacts compliant organization	control	hypernym
assets	control	hypernym
automatic locking control	control	hypernym
automatic locking	control	hypernym
automatic locking control compliant mobile device	control	hypernym
automatic locking control compliant computer	control	hypernym
mobile device	control	hypernym
computer	control	hypernym
mobile device control	control	hypernym
computer control	control	hypernym
backup media	control	hypernym
backup storage media	control	hypernym
backup storage in-house control compliant backup media	control	hypernym
backup storage outside control compliant backup media	control	hypernym
bared window	control	hypernym
bared window control compliant section	control	hypernym
bared window section	control	hypernym
business continuity management process	control	hypernym
business continuity management process	control	concept
business continuity management	control	hypernym
business continuity management	control	concept
business continuity management process control compliant organization	control	hypernym
business continuity management process control compliant organization	control	concept
cabling security control compliant building	control	hypernym
cabling security building	control	hypernym
cabling security	control	hypernym
change control procedures control compliant control	control	hypernym
change control procedures	control	hypernym
change control	control	hypernym
clear desk control compliant organization	control	hypernym
clear desk control	control	hypernym
clear desk	control	hypernym
clock synchronization control compliant computer	control	hypernym
clock synchronization control	control	hypernym
clock synchronization	control	hypernym
cloud computing control compliant organization	control	hypernym
cloud computing control	control	hypernym
cloud computing	control	hypernym
compliance control compliance organization	control	hypernym
compliance control	control	hypernym
configuration data control compliant organization	control	hypernym
configuration data control	control	hypernym
configuration data	control	hypernym
data archiving control compliant organization	control	hypernym
data archiving control	control	hypernym
data archiving	control	hypernym
data backup control compliant organization	control	hypernym
data backup control	control	hypernym
data backup	control	hypernym
data backup storage control compliant organization	control	hypernym
data backup storage control	control	hypernym
data backup storage	control	hypernym
data breach notification control compliant organization	control	hypernym
data breach notification control	control	hypernym
data breach notification	control	hypernym
data disposal control compliant organization	control	hypernym
data disposal control	control	hypernym
data disposal	control	hypernym
data encryption software control compliant computer	control	hypernym
data encryption software control	control	hypernym
data encryption software	control	hypernym
data encryption	control	hypernym
data secrecy control compliant organization	control	hypernym
data secrecy control	control	hypernym
data secrecy	control	hypernym
data security handbook control compliant organization	control	hypernym
data security handbook control	control	hypernym
data security handbook	control	hypernym
delivery and loading areas control compliant building	control	hypernym
delivery and loading areas control	control	hypernym
delivery and loading areas	control	hypernym
disciplinary process control compliant organization	control	hypernym
disciplinary process control	control	hypernym
disciplinary process	control	hypernym
documented operational procedures control compliant organization	control	hypernym
documented operational procedures control	control	hypernym
documented operational procedures	control	hypernym
electric installation control compliant organization	control	hypernym
electric installation control	control	hypernym
electric installation	control	hypernym
electronic commerce services control compliant organization	control	hypernym
electronic commerce services control	control	hypernym
electronic commerce services	control	hypernym
electronic messaging protection control compliant organization	control	hypernym
electronic messaging protection control	control	hypernym
electronic messaging protection	control	hypernym
emergency planning control compliant organization	control	hypernym
emergency planning control	control	hypernym
emergency planning	control	hypernym
emergency recovery control compliant organization	control	hypernym
emergency recovery control	control	hypernym
emergency recovery	control	hypernym
encryption of confidential data control compliant organization	control	hypernym
encryption of confidential data control	control	hypernym
encryption of confidential data	control	hypernym
environmental conditions monitoring control compliant organization	control	hypernym
environmental conditions monitoring control control	control	hypernym
environmental conditions monitoring	control	hypernym
environmental conditions monitoring control compliant sever room	control	hypernym
environmental conditions monitoring sever room	control	hypernym
equipment identification control compliant computer	control	hypernym
equipment identification computer	control	hypernym
equipment identification control	control	hypernym
equipment identification	control	hypernym
equipment location control compliant building	control	hypernym
equipment location control	control	hypernym
equipment location	control	hypernym
exchange agreements control compliant organizations	control	hypernym
exchange agreements control	control	hypernym
exchange agreements	control	hypernym
external staff control compliant organizations	control	hypernym
external staff control control	control	hypernym
external staff	control	hypernym
external connections authentication control compliant organization	control	hypernym
external connections authentication control	control	hypernym
external connections control compliant organization	control	hypernym
external connections authentication	control	hypernym
external connections	control	hypernym
fire extinguisher control compliant section	control	hypernym
fire extinguisher control compliant server room	control	hypernym
fire extinguisher section	control	hypernym
fire extinguisher server room	control	hypernym
server room	control	hypernym
fire extinguisher control	control	hypernym
fire safety control compliant organizations	control	hypernym
fire safety control control	control	hypernym
fire safety	control	hypernym
fire suppression control compliant section	control	hypernym
fire suppression control	control	hypernym
fire suppression	control	hypernym
firewall control compliant computer	control	hypernym
firewall control	control	hypernym
firewall computer	control	hypernym
firewall	control	hypernym
firewall regulation control compliant organization	control	hypernym
firewall regulation control	control	hypernym
firewall regulation	control	hypernym
human resource development control compliant organization	control	hypernym
human resource development control	control	hypernym
human resource development	control	hypernym
human resource development policy	control	hypernym
human behavior guidelines control compliant control organization	control	hypernym
human behavior guidelines control compliant organization	control	hypernym
human behavior guidelines control	control	hypernym
human behavior guidelines policy	control	hypernym
human behavior guidelines	control	hypernym
information passing control compliant organization	control	hypernym
information passing control	control	hypernym
information passing	control	hypernym
information passing policy	control	hypernym
information security independent review control compliant organization	control	hypernym
information security independent review control	control	hypernym
information security independent review	control	hypernym
information security review	control	hypernym
information security review control	control	hypernym
information security review control compliant organization	control	hypernym
information security policy control compliant organization	control	hypernym
information security policy control	control	hypernym
information security policy	control	hypernym
information security risk awareness control compliant organization	control	hypernym
information security risk awareness	control	hypernym
information security risk awareness control	control	hypernym
internet regulation control compliant organization	control	hypernym
internet regulation control	control	hypernym
internet regulation	control	hypernym
internet regulation policy	control	hypernym
intrusion alarm system control compliant section	control	hypernym
intrusion alarm system	control	hypernym
intrusion alarm system control	control	hypernym
kensington lock control compliant computer	control	hypernym
kensington lock control	control	hypernym
kensington lock	control	concept
kensington lock	control	hypernym
lightning arrester control compliant building	control	hypernym
lightning arrester control	control	hypernym
lightning arrester	control	hypernym
locked doors control compliant organization	control	hypernym
locked doors control	hypernym
locked doors	control	hypernym
maintenance contract control compliant it component	control	hypernym
maintenance contract control	control	hypernym
maintenance contract	control	hypernym
material storing control compliant building	control	hypernym
material storing control	control	hypernym
material storing	control	hypernym
media and communication scanning control compliant organization	control	hypernym
media and communication scanning control	control	hypernym
media and communication scanning	control	hypernym
media and communication scanning control compliant organization	control	hypernym
media scanning control	control	hypernym
media scanning	control	hypernym
media scanning control compliant organization	control	hypernym
communication scanning control	control	hypernym
communication scanning	control	hypernym
communication scanning control compliant organization	control	hypernym
mobile device control compliant organization	control	hypernym
mobile device control	control	hypernym
mobile device	control	hypernym
mobile code blocking control compliant organization	control	hypernym
mobile code blocking control	control	hypernym
mobile code blocking	control	hypernym
network logging and monitoring control compliant organization	control	hypernym
network logging and monitoring control	control	hypernym
network logging and monitoring	control	hypernym
network logging control compliant organization	control	hypernym
network logging control	control	hypernym
network logging	control	hypernym
network monitoring	control	hypernym
network monitoring control	control	hypernym
network monitoring control compliant organization	control	hypernym
network routing control compliant organization	control	hypernym
network routing control	control	hypernym
network routing	control	hypernym
network segregation control compliant organization	control	hypernym
network segregation control	control	hypernym
network segregation	control	hypernym
network services agreement control compliant organization	control	hypernym
network services agreement control	control	hypernym
network services agreement	control	hypernym
network services	control	hypernym
new facilities authorization control compliant organization	control	hypernym
new facilities authorization control	control	hypernym
new facilities authorization	control	hypernym
new facilities	control	hypernym
organizational records protection control compliant organization	control	hypernym
organizational records protection control	control	hypernym
organizational records protection	control	hypernym
outsourcing control compliant organization	control	hypernym
outsourcing control	control	hypernym
outsourcing	control	hypernym
password regulation control compliant computer	control	hypernym
password regulation control	control	hypernym
password regulation	control	hypernym
personal firewall control compliant computer	control	hypernym
personal firewall control	control	hypernym
personal firewall	control	hypernym
physical access key control compliant organization	control	hypernym
physical access key control	control	hypernym
physical access key	control	hypernym
physical media transit control compliant organization	control	hypernym
physical media transit	control	hypernym
physical media transit control	control	hypernym
physical perimeter protection control compliant organization	control	hypernym
physical perimeter protection control	control	hypernym
physical perimeter protection	control	hypernym
physical perimeter protection policy	control	hypernym
placement control compliant it component	control	hypernym
placement control	control	hypernym
it component	control	hypernym
port security diagnostic and configuration control compliant organization	control	hypernym
port security diagnostic and configuration	control	hypernym
port security diagnostic and configuration control	control	hypernym
port security diagnostic control compliant organization	control	hypernym
port security diagnostic	control	hypernym
port security diagnostic control	control	hypernym
position responsibilites and deputizing control compliant organization	control	hypernym
position responsibilites and deputizing control	control	hypernym
position responsibilites and deputizing	control	hypernym
private software and hardware control compliant organization	control	hypernym
private software and hardware control	control	hypernym
private software and hardware	control	hypernym
processing validation control compliant computer	control	hypernym
processing validation control	control	hypernym
processing validation	control	hypernym
raised floor control compliant level	control	hypernym
raised floor control	control	hypernym
raised floor	control	hypernym
removable media control compliant organization	control	hypernym
removable media control	control	hypernym
removable media	control	hypernym
restrictive granting of access rights control compliant organization	control	hypernym
restrictive granting of access rights control	control	hypernym
restrictive granting of access rights	control	hypernym
risk assessment control compliance organization	control	hypernym
risk assessment	control	hypernym
risk assessment control	control	hypernym
safety door control compliant section	control	hypernym
safety door control	control	hypernym
safety door	control	hypernym
secure areas control compliant building	control	hypernym
secure areas control	control	hypernym
secure areas	control	hypernym
secure log on control compliant organization	control	hypernym
secure log on control	control	hypernym
secure log on	control	hypernym
security incidents procedures control compliant organization	control	hypernym
security incidents procedures control	control	hypernym
security incidents procedures	control	hypernym
security management framework control compliant organization	control	hypernym
security management framework control	control	hypernym
security management framework	control	hypernym
security relevant patches and updates control compliant organization	control	hypernym
security relevant patches control compliant organization	control	hypernym
security relevant updates control compliant organization	control	hypernym
security relevant patches and updates control	control	hypernym
security relevant patches control	control	hypernym
security relevant updates control	control	hypernym
security relevant patches and updates	control	hypernym
security relevant patches	control	hypernym
security relevant updates	control	hypernym
security training control compliant organization	control	hypernym
security training control	control	hypernym
security training	control	hypernym
sensitive information control compliant organization	control	hypernym
sensitive information control	control	hypernym
sensitive information	control	hypernym
session time control compliant organization	control	hypernym
session time control	control	hypernym
session time	control	hypernym
smoke detector control compliant section	control	hypernym
smoke detector control	control	hypernym
smoke detector	control	hypernym
social network control compliant organization	control	hypernym
social network control	control	hypernym
social network	control	hypernym
system capacity monitoring control compliant organization	control	hypernym
system capacity monitoring control	control	hypernym
system capacity monitoring	control	hypernym
system capacity control compliant organization	control	hypernym
system capacity control	control	hypernym
system capacity	control	hypernym
systems testing and acquisition control compliant organization	control	hypernym
systems testing and acquisition control	control	hypernym
systems testing and acquisition	control	hypernym
systems testing control compliant organization	control	hypernym
systems testing control	control	hypernym
systems testing control	hypernym
systems acquisition control compliant organization	control	hypernym
systems acquisition control	control	hypernym
systems acquisition	control	hypernym
technical vulnerabilities control compliant organization	control	hypernym
technical vulnerabilities	control	hypernym
technical vulnerabilities control	control	hypernym
teleworking control compliant organization	control	hypernym
teleworking control	control	hypernym
teleworking	control	hypernym
temperature surveillance control compliant server room	control	hypernym
temperature surveillance control	control	hypernym
temperature surveillance	control	hypernym
theft protection control compliant mobile device	control	hypernym
theft protection control compliant mobile	control	hypernym
theft protection control	control	hypernym
theft protection	control	hypernym
third party service management control compliant organization	control	hypernym
third party service management control	control	hypernym
third party service management	control	hypernym
unauthorized use prevention control compliant organization	control	hypernym
unauthorized use control compliant organization	control	hypernym
unauthorized use prevention compliant organization	control	hypernym
unauthorized use prevention control	control	hypernym
unauthorized use prevention	control	hypernym
uninterruptible power supply control compliant server room	control	hypernym
uninterruptible power supply control	control	hypernym
uninterruptible power supply	control	hypernym
user identification authentication control compliant organization	control	hypernym
user identification authentication control	control	hypernym
user identification control compliant organization	control	hypernym
user identification control	control	hypernym
user identification	control	hypernym
user authentication control compliant organization	control	hypernym
user authentication control	control	hypernym
user authentication	control	hypernym
user registration and deregistration control compliant organization	control	hypernym
user registration and deregistration control	control	hypernym
user registration and deregistration	control	hypernym
user registration control compliant organization	control	hypernym
user registration control	control	hypernym
user registration	control	hypernym
virus infection control compliant organization	control	hypernym
virus infection control	control	hypernym
virus infection	control	hypernym
visible identification control compliant building	control	hypernym
visible identification control	control	hypernym
visible identification	control	hypernym
visitors control compliant organization	control	hypernym
visitors control	control	hypernym
visitors	control	hypernym
water alarm system control compliant section	control	hypernym
water alarm system control	control	hypernym
water alarm system	control	hypernym
wlan control compliant organization	control	hypernym
wlan control	control	hypernym
wlan	control	hypernym"""

for l in control.split("\n"):
    try:
        if l.split("\t")[1]!="control":
            print (l)
    except:
        print (l)
# [l for l in control if l.split("\t")[1]!="control"]

antivirus	software	hypernym
anti virus	software	hypernym
locked doors control	hypernym
systems testing control	hypernym


In [5]:
for f in incomplete_files:
    !rm ../junk/Files/$f

In [11]:
dataset = open("../files/dataset/dataset.tsv").read().split("\n")
dataset_t = [l for l in dataset if l.split("\t")[-1]!="none"]
dataset_f = [l for l in dataset if l.split("\t")[-1]=="none"]
import random
dataset_t_mini = random.choices(dataset_t, k=int(0.1 * len(dataset_t)))
dataset_f_mini = random.choices(dataset_f, k=10000-int(0.1 * len(dataset_t)))
dataset_mini = dataset_t_mini + dataset_f_mini
random.shuffle(dataset_mini)
open("../files/dataset/dataset_mini.tsv","w+").write("\n".join(dataset_mini))
open("../files/dataset/train_mini.tsv","w+").write("\n".join(dataset_mini[:int(0.9*len(dataset_mini))]))
open("../files/dataset/test_mini.tsv","w+").write("\n".join(dataset_mini[int(0.9*len(dataset_mini)):]))

33268

In [26]:
from pronto import Ontology
import re

def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return " ".join([m.group(0).lower() for m in matches])

ont = Ontology("/Users/vivek/SIREN-Research/securityontology.owl")
ontology = [camel_case_split(a.id.split("#")[-1])+"\tcontrol\thypernym" for a in ont.terms()]
import itertools

ontology = list(itertools.chain.from_iterable(itertools.repeat(x, 3) for x in ontology))

In [46]:
lines = [l for l in open("temp_policy","r").read().split("\n") if "policy" in l.split("\t")[0]]
final = []
for l in lines:
    final.append(l.split("\t")[0] + "\tpolicy\thypernym")
    final.append(" ".join(l.split("\t")[0].split()[:-1]) + "\tpolicy\thypernym")
    final.append(l.split("\t")[0] + "\tintangible asset\thypernym")
    final.append(" ".join(l.split("\t")[0].split()[:-1]) + "\tintangible asset\thypernym")
    final.append(l.split("\t")[0] + "\tasset\thypernym")
    final.append(" ".join(l.split("\t")[0].split()[:-1]) + "\tasset\thypernym")
    

In [47]:
open("../junk/policy","w+").write("\n".join(final))

15077

In [52]:
lines = [l.split("\t") for l in open("../files/dataset/test_knocked.tsv","r").read().split("\n")]
final_lines = []
for l in lines:
    final_lines.append("\t".join(l))
    final_lines.append("\t".join(l[:2][::-1] + [l[2].replace("concept", "instance").replace("hypernym", "hyponym")]))
final_lines = sorted(list(set(final_lines)))
open("../files/dataset/test_knocked.tsv","w+").write("\n".join(final_lines))

287258

In [51]:
len(final_lines), len(list(set(final_lines)))

(5910, 5562)